In [1]:
import pandas as pd
import numpy as np

# !pip install deepctr[gpu]

# 导入deepctr
from deepctr.models import DIN,DIEN,BST
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
import tensorflow as tf

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

generate='./data/generate/'
path='./data/rec_data/'

2022-05-30 20:59:56.013646: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
user_seq=pd.read_csv(generate+'user_seq.csv',usecols=['suv','itemId','length'])
user_seq.rename(columns={'length':'seq_length'},inplace=True)
train=pd.read_csv(path+'train-dataset.csv')
test=pd.read_csv(path+'rec-train-dataset.csv')
test['label']=9
category=pd.read_csv(generate+'category14.csv')
category.drop_duplicates(inplace=True)
train=pd.concat([train,test])
del test
train=pd.concat([train,user_seq])
train=pd.merge(train,category,how='left',on='itemId')

In [3]:
# 数据处理sparse、dense
for feat in ['itemId']:
    lbe = LabelEncoder()
    train[feat] = lbe.fit_transform(train[feat])
    
train[['itemId']]+=1
# train[['category']]+=1
    
ygq=train.copy()
user_seq=train[train['sampleId'].isna()]
user_seq=user_seq[['suv','itemId','category','seq_length']]
train=train[train['sampleId'].notna()]
del train['seq_length']

In [4]:
# 造点统计特征
train['pvid_count'] = train.groupby('pvId')['itemId'].transform('count')
train['pvid_item_nunique'] = train.groupby('pvId')['itemId'].transform('nunique')
train['pvid_suv_nunique'] = train.groupby('pvId')['suv'].transform('nunique')
train['item_count'] = train.groupby('itemId')['itemId'].transform('count')
train['item_suv_nunique'] = train.groupby('itemId')['suv'].transform('nunique')

In [5]:
# train=train[train['userSeq'].notna()]
del train['userSeq']
del train['logTs']

In [6]:
user_seq

,suv,itemId,category,seq_length
5632823,1640622537887022,126273,8,42.0
5632824,1640889643810cd2xc4,115818,11,21.0
5632825,1641397505962MKVTgo,121921,1,15.0
5632826,1625501468892nls973,125218,11,21.0
5632827,1625501468892nls973,126528,11,21.0
...,...,...,...,...
12764265,1609112059256284,33141,14,51.0
12764266,1609112059256284,33130,11,51.0
12764267,1609112059256284,33114,11,51.0
12764268,1609112059256284,33131,11,51.0


In [7]:
hist_click=user_seq[['suv','itemId','category','seq_length']].groupby(['suv','seq_length']).agg({list}).reset_index()
his_behavior_df=pd.DataFrame()
his_behavior_df['suv']=hist_click['suv']
his_behavior_df['seq_length']=hist_click['seq_length']
his_behavior_df['hist_itemId']=hist_click['itemId']
his_behavior_df['hist_category']=hist_click['category']
del hist_click
del user_seq

In [8]:
data=pd.merge(train,his_behavior_df,how='left',on='suv')

In [9]:
tmps=data['hist_itemId'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
data['hist_itemId']=tmps

tmps=data['hist_category'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
data['hist_category']=tmps

# data['length'].fillna(0,inplace=True)

del data['sampleId']

In [10]:
# 把特征分开
sparse_fea = ['suv', 'pvId','itemId', 'operator','category',
              'browserType', 'deviceType', 'osType', 'province', 'city']

behavior_fea = ['itemId','category']

hist_behavior_fea = ['hist_itemId','hist_category']

dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_count', 'item_suv_nunique']
# dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_suv_nunique']

In [11]:
sparse_fea_tmp = ['suv', 'pvId', 'operator',
              'browserType', 'deviceType', 'osType', 'province', 'city']

# 数据处理sparse、dense
for feat in sparse_fea_tmp:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

data[dense_fea] = data[dense_fea].fillna(0, )
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_fea] = mms.fit_transform(data[dense_fea])

In [12]:
data.to_feather(generate+'DIN_data.feather')
ygq.to_feather(generate+'DIN_data_ygq.feather')

==============================================================================

In [13]:
import pandas as pd
import numpy as np

# !pip install deepctr[gpu]

# 导入deepctr
from deepctr.models import DIN,DIEN,BST
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
import tensorflow as tf

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

generate='./data/generate/'
path='./data/rec_data/'

In [14]:
# 把特征分开
sparse_fea = ['suv', 'pvId','itemId', 'operator','category',
              'browserType', 'deviceType', 'osType', 'province', 'city']

behavior_fea = ['itemId','category']

hist_behavior_fea = ['hist_itemId','hist_category']

dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_count', 'item_suv_nunique']

In [15]:
data=pd.read_feather(generate+'DIN_data.feather')
ygq=pd.read_feather(generate+'DIN_data_ygq.feather')
data['seq_length'].fillna(0,inplace=True)
# data['seq_length']=data['seq_length'].astype('float32')

In [16]:
emb_dim=32
max_len=95

sparse_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique() + 1, embedding_dim=emb_dim) for feat in sparse_fea]
dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_fea]

var_feature_columns = [VarLenSparseFeat(SparseFeat(feat, vocabulary_size=ygq[feat.split('_')[1]].nunique() + 1,
                        embedding_dim=emb_dim, embedding_name=feat.split('_')[1]), maxlen=max_len,length_name="seq_length") for feat in hist_behavior_fea]

dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns


In [17]:
train=data[data['label']!=9]
test=data[data['label']==9]
del data
del ygq
test.head(20)

,label,pvId,suv,itemId,operator,browserType,deviceType,osType,province,city,category,pvid_count,pvid_item_nunique,pvid_suv_nunique,item_count,item_suv_nunique,seq_length,hist_itemId,hist_category
4233804,9.0,5,325430,119878,266,26,0,9,32,187,8,0.041667,0.045016,0.0,0.087105,0.088580,0.0,[0],[0]
4233805,9.0,5,325430,122049,266,26,0,9,32,187,11,0.041667,0.045016,0.0,0.386258,0.392861,0.0,[0],[0]
4233806,9.0,5,325430,123015,266,26,0,9,32,187,11,0.041667,0.045016,0.0,0.171289,0.174535,0.0,[0],[0]
4233807,9.0,5,325430,105010,266,26,0,9,32,187,11,0.041667,0.045016,0.0,0.423353,0.427223,0.0,[0],[0]
4233808,9.0,5,325430,123783,266,26,0,9,32,187,11,0.041667,0.045016,0.0,0.208711,0.212068,0.0,[0],[0]
4233809,9.0,5,325430,109509,266,26,0,9,32,187,8,0.041667,0.045016,0.0,0.454743,0.459098,0.0,[0],[0]
4233810,9.0,5,325430,122362,266,26,0,9,32,187,1,0.041667,0.045016,0.0,0.430615,0.437152,0.0,[0],[0]
4233811,9.0,5,325430,116553,266,26,0,9,32,187,14,0.041667,0.045016,0.0,0.574558,0.582410,0.0,[0],[0]
4233812,9.0,5,325430,119117,266,26,0,9,32,187,14,0.041667,0.045016,0.0,0.103618,0.105387,0.0,[0],[0]
4233813,9.0,5,325430,115665,266,26,0,9,32,187,11,0.041667,0.045016,0.0,0.077438,0.078966,0.0,[0],[0]


In [18]:
x_trn = {}
for name in get_feature_names(dnn_feature_columns):
    if name in hist_behavior_fea:
        # 这是历史行为序列
        his_list = [l for l in train[name]]
        x_trn[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组
    else:
        x_trn[name] = train[name].values

x_tst = {}
for name in get_feature_names(dnn_feature_columns):
    if name in hist_behavior_fea:
        # 这是历史行为序列
        his_list = [l for l in test[name]]
        x_tst[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组
    else:
        x_tst[name] = test[name].values

In [19]:
y_trn=train['label'].values
y_tst=test['label'].values
# y_val=val['label'].values

In [20]:
# 建立模型
model = BST(dnn_feature_columns, behavior_fea,dnn_dropout=0.1)

# 查看模型结构
model.summary()

# 模型编译
model.compile('adam', 'binary_crossentropy',metrics=['binary_crossentropy','accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
itemId (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
category (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
hist_itemId (InputLayer)        [(None, 95)]         0                                            
__________________________________________________________________________________________________
hist_category (InputLayer)      [(None, 95)]         0                                            
______________________________________________________________________________________________

In [21]:
# history = model.fit(x_trn, y_trn, verbose=1, epochs=10, validation_split=0.3,batch_size=256)
history = model.fit(x_trn, y_trn, verbose=1, epochs=1,batch_size=256)

Train on 4233804 samples


2022-05-30 21:02:07.056900: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-30 21:02:07.094801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA RTX A5000 major: 8 minor: 6 memoryClockRate(GHz): 1.695
pciBusID: 0000:61:00.0
2022-05-30 21:02:07.094820: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-30 21:02:07.098454: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-05-30 21:02:07.100233: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-05-30 21:02:07.100456: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2022-05-30 21:02:07.100864: I tensorflow/stream_executor/platform/defaul

4233804/4233804 [==============================] - 473s 112us/sample - loss: 0.4720 - binary_crossentropy: 0.4625 - acc: 0.8123


In [22]:
# 模型预测
test['rate'] = model.predict(x_tst, verbose=1, batch_size=256)
# test[['suv', 'itemId', 'rate']].to_csv('./din_rank_score.csv', index=False)

   3072/1399019 [..............................] - ETA: 1:58

2022-05-30 21:10:01.420690: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 0, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-30 21:10:01.421969: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 1, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-30 21:10:01.423221: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] layout failed: Invalid argument: The graph couldn't be sorted in topological order.
2022-05-30 21:10:01.425527: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] model_pruner failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'attention_sequence_pooling_layer/local_activation_unit/concat' has self cycle fanin 'attention_sequence_pooling_layer/local_activation_unit/concat'.
2022-05-30 21:10:01.432315: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] remapper failed

1399019/1399019 [==============================] - 53s 38us/sample


In [23]:
label_raw=pd.read_csv(path+'rec-train-dataset.csv',usecols=['label'])
label_raw=label_raw['label'].tolist()
test['label']=label_raw

In [24]:
ygq=test[['label','pvId','rate']]

In [25]:
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm import tqdm
def uAUC(labels, preds, pvId_list):
    user_pred = defaultdict(lambda: [])
    user_truth = defaultdict(lambda: [])
    for idx, truth in enumerate(labels):
        pvId = pvId_list[idx]
        pred = preds[idx]
        truth = labels[idx]
        user_pred[pvId].append(pred)
        user_truth[pvId].append(truth)
    total_auc = 0.0
    size = 0.0
    
    pvId_flag = defaultdict(lambda: False)
    for pvId in set(pvId_list):
        truths = user_truth[pvId]
        flag = False
        for i in range(len(truths) - 1):
            if truths[i] != truths[i + 1]:
                flag = True
                break
        pvId_flag[pvId] = flag

    for pvId in tqdm(set(pvId_list)):
        if pvId_flag[pvId]:
            auc = roc_auc_score(np.asarray(user_truth[pvId]), np.asarray(user_pred[pvId]))
            total_auc += auc 
            size += 1.0
    # size = max(size, 1)
    user_auc = float(total_auc)/size
    return user_auc

In [26]:
print(uAUC(ygq['label'].values.tolist(), ygq['rate'].values.tolist(), ygq['pvId'].values.tolist()))

100%|██████████| 100106/100106 [01:02<00:00, 1612.07it/s]


0.5913209569985907
